In [1]:
## Technologies
# CUDA toolkit 10.0
# Tensorflow-gpu 2.0
# cuDNN 7.6.4

In [2]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image, ImageFile

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6040)

VirtualDeviceConfiguration(memory_limit=6040)

In [3]:
#map betweem image names and labels 
matData = loadmat('./cars_annos.mat')

labelsList = []
for arr in matData['annotations'][0]:
    imgPath = arr[0][0]
    labelNum = arr[5][0][0]
    splitTag = arr[6][0][0]
    labelsList.append([imgPath, labelNum, splitTag])
print("img nummber: ", len(labelsList))

labelNameDict = {}
labelList = []
for i,arr in enumerate(matData['class_names'][0]):
    labelName = arr[0]
    labelNameDict[i+1] = labelName
    labelList.append(labelName)
print("label number: ", len(labelNameDict))

labelsDF = pd.DataFrame(labelsList, columns=['imgPath', 'label', 'testTag'])
labelsDF['label'] = labelsDF['label'].map(labelNameDict)
labelsDF.tail(10)

img nummber:  16185
label number:  196


,imgPath,label,testTag
16175,car_ims/016176.jpg,smart fortwo Convertible 2012,1
16176,car_ims/016177.jpg,smart fortwo Convertible 2012,1
16177,car_ims/016178.jpg,smart fortwo Convertible 2012,1
16178,car_ims/016179.jpg,smart fortwo Convertible 2012,1
16179,car_ims/016180.jpg,smart fortwo Convertible 2012,1
16180,car_ims/016181.jpg,smart fortwo Convertible 2012,1
16181,car_ims/016182.jpg,smart fortwo Convertible 2012,1
16182,car_ims/016183.jpg,smart fortwo Convertible 2012,1
16183,car_ims/016184.jpg,smart fortwo Convertible 2012,1
16184,car_ims/016185.jpg,smart fortwo Convertible 2012,1


In [4]:
#split train and test
trainDF = labelsDF[labelsDF['testTag']==0]
print("amount of train set:", len(trainDF))
testDF = labelsDF[labelsDF['testTag']==1]
print("amount of test set:", len(testDF))
validationDF = testDF.sample(n=1000)
print("amount of validationDF:", len(validationDF))

amount of train set: 8144
amount of test set: 8041
amount of validationDF: 1000


In [9]:
#model parms
classNum = len(labelNameDict)
classContent = labelList

In [8]:
#base model
base_model = tf.keras.applications.resnet.ResNet50(weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)

In [9]:
#add top layer
top = base_model.output
top = tf.keras.layers.GlobalAveragePooling2D()(top)
top = tf.keras.layers.Flatten()(top)
top = tf.keras.layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001))(top)
top = tf.keras.layers.BatchNormalization()(top)
top = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001))(top)
top = tf.keras.layers.BatchNormalization(name='bn_fc_01')(top)
top_model = tf.keras.layers.Dense(classNum, activation='softmax')(top)

In [10]:
# learning rate
# adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [11]:
#create model
model = tf.keras.models.Model(inputs=base_model.input, outputs=top_model)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
#model load
model = tf.keras.models.load_model('models/3/car.15-0.17.hdf5')

In [ ]:
#model config
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['acc'])

In [16]:
#call back functions
early_stop = tf.keras.callbacks.EarlyStopping('acc', patience=5)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('./models/car.{epoch:02d}-{val_acc:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau('loss', factor=0.1, patience=2)
callbacks = [early_stop, model_checkpoint, reduce_lr]

In [6]:
#data preprocess parms
img_width, img_height = 224, 224
batch_size = 10
epochs = 5
trainNum = 8000
validNum = len(validationDF)

In [10]:
#input data preprocess
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True
)
train_generator = train_datagen.flow_from_dataframe(
    trainDF.sample(n=trainNum),
    target_size = (img_width, img_height),
    x_col = 'imgPath',
    y_col = 'label',
    classes = classContent,
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 8000 validated image filenames belonging to 196 classes.


In [11]:
#valid data preprocess
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
valid_generator = valid_datagen.flow_from_dataframe(
    validationDF,
    target_size = (img_width, img_height),
    x_col = 'imgPath',
    y_col = 'label',
    classes = classContent,
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 1000 validated image filenames belonging to 196 classes.


In [ ]:
#train model
model_history = model.fit_generator(
    train_generator,
    steps_per_epoch = trainNum / batch_size,
    validation_data = valid_generator,
    validation_steps = validNum / batch_size,
    epochs = epochs,
    callbacks = callbacks,
    verbose=1
)

Epoch 1/25
799/800 [============================>.] - ETA: 0s - loss: 1.1363 - acc: 0.8507
Epoch 00001: val_acc did not improve from 0.21300
800/800 [==============================] - 210s 263ms/step - loss: 1.1366 - acc: 0.8508 - val_loss: 6.8155 - val_acc: 0.2030
Epoch 2/25
799/800 [============================>.] - ETA: 0s - loss: 1.1342 - acc: 0.8487
Epoch 00002: val_acc did not improve from 0.21300
800/800 [==============================] - 211s 264ms/step - loss: 1.1341 - acc: 0.8487 - val_loss: 6.7035 - val_acc: 0.2050
Epoch 3/25
799/800 [============================>.] - ETA: 0s - loss: 1.1183 - acc: 0.8522
Epoch 00003: val_acc did not improve from 0.21300
800/800 [==============================] - 206s 258ms/step - loss: 1.1185 - acc: 0.8521 - val_loss: 6.7778 - val_acc: 0.2020
Epoch 4/25
799/800 [============================>.] - ETA: 0s - loss: 1.1162 - acc: 0.8577
Epoch 00004: val_acc did not improve from 0.21300
800/800 [==============================] - 205s 256ms/step - 